## Imports for project

In [71]:
import pandas as pd
from sklearn import preprocessing

## Import data from csv

In [72]:
data = pd.read_csv('all_stocks_5yr.csv')

## Data Encoding

In [73]:
#cut off year
data['date'] = data['date'].astype(str).str[:-5]
#Encode data for Date and Name
le = preprocessing.LabelEncoder()
data["date"] = le.fit_transform(data["date"])
data["Name"] = le.fit_transform(data["Name"])



## Data Normalization

In [74]:
#data['date'] = data["date"].astype(np.float64)
#data['volume'] = data["volume"].astype(np.float64)
#data['Name'] = data["Name"].astype(np.float64)
#print(data.info())
#print("start---")
#count =0
#for i in data:
#    if data.loc[[count]].isnull().values.any():
#        print(data.loc[[count]])    
#    count += 1
#print("done---")
#data_norm = preprocessing.normalize(data,norm="l2")
#print(data_norm)

## Cluster Data on dates

In [171]:
cdata = data.copy()
dates = list(range(0,363))
first2 = True
#col = ['date','open','high','low','close','volume','Name']
#dates2 = pd.DataFrame(columns=col)
#loop through all unique days present
for day in dates:
    #loop though and compare for all matching days and add to list
    first = True
    for i in range(cdata.__len__()):
        if cdata.iloc[i,0] == day:
            if first == True:
                #print(pd.DataFrame(cdata.iloc[[i]]))
                dates2 = pd.DataFrame(cdata.iloc[[i]])
                #print(dates2)
                first = False
            else:    
                dates2 = dates2.append(pd.DataFrame(cdata.iloc[[i]]))
    #take mean of list
    if first2 == True:
        series = dates2.mean(axis=0)
        df = pd.DataFrame(series).transpose()
        #print(type(df))
        #print(df)
        average = pd.DataFrame(df)
        #print(average)
        first2 = False
    else:
        #print(type(average))
        series = dates2.mean(axis=0)
        df = pd.DataFrame(series).transpose()
        average = average.append(pd.DataFrame(df))
   #if day==3:
        #print(type(dates2))
        #print(dates2)
        #break


In [172]:
type(average)
test = average.copy()
test.transpose()
print(test)

     date        open        high         low       close        volume  \
0     0.0   91.509084   92.259729   90.723094   91.572180  4.213415e+06   
0     1.0   93.489742   94.364573   92.457010   93.577147  4.644509e+06   
0     2.0   90.942172   91.684775   89.839074   90.915487  4.398161e+06   
0     3.0   81.307161   82.017693   79.799221   80.321716  4.933475e+06   
0     4.0   76.367409   77.616414   75.561878   77.067580  5.456638e+06   
0     5.0   76.521747   77.420899   75.593904   76.431084  6.070596e+06   
0     6.0   88.615370   89.577656   87.734597   88.699235  5.005569e+06   
0     7.0   92.249161   93.066075   91.430644   92.329622  4.887149e+06   
0     8.0  101.843556  102.574246  100.944295  101.825049  4.268157e+06   
0     9.0   93.796959   94.533200   92.620087   93.501093  5.036600e+06   
0    10.0   87.706329   88.437654   86.699488   87.578221  3.527404e+06   
0    11.0   82.829737   83.705149   81.473333   82.867071  5.880918e+06   
0    12.0   76.687494   7

## Split Dataset

In [174]:
training = pd.DataFrame(test.sample(frac=0.8,replace=False))
test = pd.DataFrame(pd.concat([test,training]).drop_duplicates(keep=False))

## Logistic Regression